In [8]:
# Step 0: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
# Step 1: Imports
import sys, os, datetime, numpy as np
sys.path.append('/content/drive/MyDrive/Colab Notebooks/Deep_Learning/mura_modules')

import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from tensorflow.keras import mixed_precision

from mura_dataset_tfdata import get_mura_dataset_binary_only_augmented
from config import BASE_PATH_GOOGLE_DRIVE, IMAGE_SIZE, BATCH_SIZE, NUM_CLASSES_BINARY
from model_pretrained_binary import build_binary_pretrained_model, fine_tune_binary_model
from metrics import get_binary_metrics
from plot_utils import plot_training_history

In [10]:
# Step 2: Set precision
mixed_precision.set_global_policy("mixed_float16")

In [11]:
# Step 3: Paths
train_csv = BASE_PATH_GOOGLE_DRIVE + 'train_image_metadata.csv'
valid_csv = BASE_PATH_GOOGLE_DRIVE + 'valid_image_metadata.csv'

In [12]:
# Step 4: Data pipeline
train_ds = get_mura_dataset_binary_only_augmented(
    csv_path=BASE_PATH_GOOGLE_DRIVE + "train_image_metadata.csv",
    training=True,
    cache_dir="/content"
)

valid_ds = get_mura_dataset_binary_only_augmented(
    csv_path=BASE_PATH_GOOGLE_DRIVE + "valid_image_metadata.csv",
    training=False,
    cache_dir="/content"
)


In [13]:
# Step 5: Build model (frozen base)
model, base_model  = build_binary_pretrained_model(
    input_shape=IMAGE_SIZE + (3,),
    num_classes_binary=NUM_CLASSES_BINARY
)

16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [14]:
# Step 6: Callbacks
callbacks = [
    ModelCheckpoint("best_binary_model.keras", monitor="val_loss", save_best_only=True),
    EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True),
    ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, min_lr=1e-6),
    TensorBoard(log_dir=f"logs/fit/{datetime.datetime.now().strftime('%Y%m%d-%H%M%S')}", histogram_freq=1)
]


In [15]:
# Step 7: Clear previous cache lock (if any)
lockfile_path = "/content/mura_train_cache.tf-data_0.lockfile"
if os.path.exists(lockfile_path):
    os.remove(lockfile_path)

In [ ]:
import datetime
from tensorflow.keras.callbacks import ModelCheckpoint

# Step 8: Train (frozen)
timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M')
checkpoint_path = f"/content/drive/MyDrive/Colab Notebooks/Deep_Learning/binary_model_frozen_augmented_{timestamp}.h5"

checkpoint_cb = ModelCheckpoint(
    filepath=checkpoint_path,
    monitor="val_loss",
    save_best_only=True,
    save_weights_only=False  # Saves entire model
)

history = model.fit(
    train_ds,
    validation_data=valid_ds,
    epochs=10,
    callbacks=[checkpoint_cb],
    verbose=1
)


Epoch 1/10
1109/1151 ━━━━━━━━━━━━━━━━━━━━ 7:43 11s/step - accuracy: 0.6557 - loss: 0.6443 - precision: 0.6279 - recall: 0.7905

In [ ]:
# Step 9: Fine-tune (unfreeze)
model.get_layer("efficientnetb0").trainable = True  # Unfreeze the base model

# Recompile with lower learning rate
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

In [ ]:
# Step 10: Continue training (fine-tuned)
timestamp_fine = datetime.datetime.now().strftime('%Y%m%d_%H%M')
checkpoint_path_fine = f"/content/drive/MyDrive/Colab Notebooks/Deep_Learning/binary_model_finetuned_augmented_{timestamp_fine}.h5"

checkpoint_cb_fine = ModelCheckpoint(
    filepath=checkpoint_path_fine,
    monitor="val_loss",
    save_best_only=True
)

history_fine = model.fit(
    train_ds,
    validation_data=valid_ds,
    epochs=20,
    callbacks=[checkpoint_cb_fine],
    verbose=1
)


In [ ]:
# Step 11: Evaluate
y_true, y_pred = [], []
for batch_x, y_true_batch in valid_ds:
    preds = model.predict(batch_x, verbose=0)
    y_true.extend(y_true_batch.numpy())
    y_pred.extend(preds)

metrics = get_binary_metrics(np.array(y_true), np.array(y_pred))
print("\n=== Final Binary Classification Report ===")
for k, v in metrics.items():
    print(f"{k}: {v:.4f}")


In [ ]:

# Step 12: Plot
plot_training_history(history)
plot_training_history(history_fine, label_prefix="Fine-tune")